In [1]:
from cvxpy_portfolio_optimizer.objective_function import (
    CVaRObjectiveFunction,
    VarianceObjectiveFunction,
    ExpectedReturnsObjectiveFunction,
)
from cvxpy_portfolio_optimizer.constraint_function import SumToOneConstraint, NoShortSellConstraint
from cvxpy_portfolio_optimizer.portfolio_optimization_problem import PortfolioOptimizationProblem
import yfinance as yf

In [2]:
tickers = ["TSLA", "MSFT", "IBM", "GOOG", "AAPL", "AMZN", "ADBE"]
rets = (
    yf.download(tickers, period="1y")["Adj Close"].loc[:, tickers].pct_change().iloc[1:, :].ffill()
)
rets

[*********************100%***********************]  7 of 7 completed


,TSLA,MSFT,IBM,GOOG,AAPL,AMZN,ADBE
Date,,,,,,,
2022-05-24,-0.069255,-0.003952,0.020050,-0.051407,-0.019216,-0.032141,-0.020528
2022-05-25,0.048777,0.011170,0.004410,-0.000817,0.001140,0.025696,0.010266
2022-05-26,0.074271,0.012875,0.018603,0.023210,0.023199,0.040295,0.015155
2022-05-27,0.073333,0.027604,0.017386,0.041581,0.040757,0.036632,0.048018
2022-05-31,-0.001804,-0.005014,-0.003088,0.010993,-0.005346,0.043970,-0.027416
...,...,...,...,...,...,...,...
2023-05-16,0.001022,0.007368,0.000811,0.026761,0.000000,0.019784,-0.001620
2023-05-17,0.044079,0.009452,0.018225,0.011575,0.003603,0.018519,0.033381
2023-05-18,0.017428,0.014395,0.003500,0.016793,0.013666,0.022944,0.010655


In the following portfolio optimization problem we will:
- Minimize CVaR at 95% confidence level
- Minimize Variance
- Maximize Expected Retruns

In [3]:
pop = PortfolioOptimizationProblem(
    returns=rets,
    objective_functions=[
        CVaRObjectiveFunction(confidence_level=0.95, weight=1.0),
        VarianceObjectiveFunction(weight=1.0),
        ExpectedReturnsObjectiveFunction(weight=0.3),
    ],
    constraint_functions=[
        SumToOneConstraint(),
        NoShortSellConstraint(),
    ],
)

In [5]:
ptf = pop.solve(solver="ECOS")

In [6]:
ptf.weights

TSLA    0.000000
MSFT    0.338102
IBM     0.479004
GOOG    0.000000
AAPL    0.182895
AMZN    0.000000
ADBE    0.000000
dtype: float64

In [7]:
ptf.objective_values

[{<ObjectiveFunctionName.CVAR: 'CVaR'>: 0.029028499211609417},
 {<ObjectiveFunctionName.VARIANCE: 'VARIANCE'>: 0.04999096599731992},
 {<ObjectiveFunctionName.EXPECTED_RETURNS: 'EXPECTED_RETURNS'>: -0.04792116480264919}]